In [1]:
using TensorDecomposition, Symbolics, LinearAlgebra, Combinatorics, IterTools

In [2]:
function kronMat(A::Matrix, d)
    if d == 1
        B = A
    else
        n, r = size(A)
        B = zeros(eltype(A), (n^d, r))
        for i=1:r 
            B[:, i] = kron(ntuple(x->A[:, i], d)...)
        end;
    end;
    return B
end;

function kronMat2(A, d)
    n, _ = size(A)
    A_ = kronMat(A, d)
    inds = with_replacement_combinations(1:n, d)
    inds = map(x->from_multiindex(x, n), inds)
    return A_[inds, :]
end;

function makeRankedTensor2(L, A, d)
    T = makeRankedTensor(L, A, d)
    T[abs.(T) .<= 1e-14] .= 0
end

function from_multiindex(x, n)
    d = length(x)
    c = 0
    for i=1:d-1
        c += (x[i]-1)*n^(d-i)
    end
    return c + x[d]
end;

function rkF(d)
    return 2^(d-1)
end

function rkAk(d, k)
    c = 0
    for i=0:k
       c += binomial(d-1, i) 
    end
    return c
end

function rkCatk(d, k)
    return binomial(d, k)
end;

In [3]:
d = 4;
r = rkF(d);

In [4]:
T = zeros(repeat([d], d)...)
for perm in permutations(1:d)
    T[perm...] = 1
end

In [5]:
P = zeros(d, r)
P[1, :] = ones(r)
sigs = product(repeat([[-1, 1]], d-1)...)
for (i, sig) in enumerate(sigs)
    P[2:end, i] .= sig
end

LT = randn(d, d);

In [6]:
a = Symbolics.variables(:a, 1:d-1)

A = repeat([append!([Num(1)], [a[i] for i=1:d-1])], 2^(d-1))
A = reduce(hcat, A) .* P;

In [7]:
# as = (im^(1/5))*collect(2:d);
as = randn(d-1)

Asub = Symbolics.value.(Symbolics.substitute(A, Dict([a[i]=> as[i] for i=1:d-1])))
L = prod.(eachcol(P)) ./ (r*prod(as));
Td = makeRankedTensor(L, Asub, d);

# Asub2 = permutedims((Complex.(L) .^ (1/d))) .* Asub;
function Asubd(L, Asub, d)
    return permutedims((Complex.(L) .^ (1/d))) .* Asub;
end;

In [8]:
x = Symbolics.variables(:x, 0:d-1)
L_ = prod.(eachcol(P));
F = Symbolics.simplify(Symbolics.expand(sum(sum(eachrow(A .* x)) .^ (d) .* L_))/ (r* prod(a)))

24.0x₀*x₁*x₂*x₃

In [9]:
B = LT*Asubd(L, Asub, d)
B1 = B[1, :]
LB = B1 .^ d
B ./= permutedims(B1);

In [10]:
TB = makeRankedTensor(LB, B, d);
FB = sum(sum(eachrow(B .* x)) .^ (d) .* LB);

In [139]:
TBhank = hankMat(real.(TB))

35×35 Matrix{HomotopyContinuation.ModelKit.Expression}:
    50.5447994418856      24.21773328054  …  -0.674022560550836
      24.21773328054   -12.4510257751436                    h₅₀
    21.0787895214743    1.21793775373069                    h₅₅
   0.148700256793748   -5.23773167662349                    h₅₆
   -12.4510257751436   -40.8344293099627                    h₇₁
    1.21793775373069   -14.9819966748973  …                 h₇₆
   -5.23773167662349   0.399949385569873                    h₇₇
    4.89634585809816   -4.57772805187065                    h₈₂
   -2.62870098599402  -0.899755716398148                    h₈₃
   -4.77993884669587    2.82591049574544                    h₈₄
   -40.8344293099627   -35.7876102189222  …                 h₉₉
   -14.9819966748973   -11.7574525421153                   h₁₀₄
   0.399949385569873    10.6588524774269                   h₁₀₅
                   ⋮                      ⋱  
   -2.96099003701262                 h₃₉                   h₁₄₆
  